In [37]:
from utils import setup_env_path

setup_env_path()

%load_ext autoreload
%autoreload 2

Former working directory:  /Users/ambroisebertin/Desktop/prog/prog_abeilles/fil-rouge-pollinisateurs
Current working directory:  /Users/ambroisebertin/Desktop/prog/prog_abeilles/fil-rouge-pollinisateurs
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import pandas as pd
import numpy as np

In [39]:
API_keys = [
    "2b10Me1HF0rfjoGWCseolNa8e",
    "2b10Xs7brWPuBdRTSeWx9V7HJu",
    "2b10umm8L2jTYjlWrJKPfshJ0O",
    "2b10sEg5pSsrtVT372XqlrzcLe",
    "2b10LDWWlXJwuS6FleaqfK6Rke",
]

## Préparation des données

In [40]:
# Import spipoll.csv as a pandas dataframe
spipoll = pd.read_csv("././data/spipoll.csv",low_memory=False)

# Extract relevant columns from the spipoll dataframe
plantes = spipoll[['collection_id', 'plante_sc', 'plante_fr',
       'plante_precision', 'plante_inconnue', 'plante_caractere',
       'photo_fleur', 'photo_plante', 'photo_feuille']]

# Shrinking the data by grouping by collection_id
# Keep only the first row for each unique value of collection_id
plantes = plantes.drop_duplicates(subset='collection_id', keep='first')

plantes.shape

(75372, 9)

In [41]:
values_to_check = ["Je ne sais pas", "Plante inconnue"]

In [42]:
plantes_sc_unlabelled = plantes.loc[ #renommer en plantes_unlabelled_sc pour plus de clarté ??
    (
        (plantes["plante_sc"].isna() | plantes["plante_sc"].isin(values_to_check))
    )
    | ((plantes["plante_inconnue"] == 1.0))
]

# creation du dataframe contenant les valeurs de plantes mais pas de plantes_sc_unlabelled
plantes_sc_labelled = plantes[~plantes.index.isin(plantes_sc_unlabelled.index)]

# in plante add "1" to the column group for each row that is in plantes_sc_labelled
plantes.loc[plantes_sc_labelled.index, 'group'] = 1

plantes_sc_labelled.shape

(63465, 9)

In [43]:
# verification des dimensions des deux df par rapport à la dim de plantes
plantes_sc_labelled.shape[0] + plantes_sc_unlabelled.shape[0] == plantes.shape[0]

True

Compression de l'information étiquettée redondante :

In [44]:
# group plantes_sc_labelled by unique values of plante_sc
plantes_sc_labelled = plantes_sc_labelled.drop_duplicates(subset='plante_sc', keep='first')

# display the dimension of the dataframe
plantes_sc_labelled.shape

(2620, 9)

On a donc un df avec 2620 plantes dont le nom est spécifié dans plante_sc et 11.907 (~1.8% des données d'origine, spipoll.csv) dont le nom n'est pas spécifié dans plante_sc.

In [45]:
# ajoute trois colonnes après plante_sc dans le dataframe plantes_sc_labelled "Famille", "Genre" et "Espèce" entre la colonne plante_sc et plante_fr
plantes_sc_labelled.insert(2, "Famille", np.nan)
plantes_sc_labelled.insert(3, "Genre", np.nan)
plantes_sc_labelled.insert(4, "Espece", np.nan)

# ajoute trois colonnes après plante_sc dans le dataframe plantes_sc_unlabelled "Famille", "Genre" et "Espèce" entre la colonne plante_sc et plante_fr
plantes_sc_unlabelled.insert(2, "Famille", np.nan)
plantes_sc_unlabelled.insert(3, "Genre", np.nan)
plantes_sc_unlabelled.insert(4, "Espece", np.nan)

In [46]:
plantes_sc_labelled.head()

,collection_id,plante_sc,Famille,Genre,Espece,plante_fr,plante_precision,plante_inconnue,plante_caractere,photo_fleur,photo_plante,photo_feuille
0,1,Les Trèfles à fleurs blanches ou roses en boul...,NaN,NaN,NaN,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
2,2,Acanthus mollis,NaN,NaN,NaN,Acanthe molle,NaN,NaN,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
5,3,"Les Orchidées à fleurs blanches, jaunes ou ver...",NaN,NaN,NaN,NaN,orchis bouc - Himantoglossum hircinum,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
6,4,Les Vipérines à fleurs bleues-violacées (des e...,NaN,NaN,NaN,NaN,Theaceae,0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
10,5,Les Pâquerettes (Bellis sp),NaN,NaN,NaN,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...


## Appel à l'API

In [47]:
from data_quality.plant_treatment.plantnet_api import PlantNetPredictor

def call_API(iloc, api_key_index=0):
    predictor = PlantNetPredictor(key = API_keys[api_key_index])
    prediction = None

    try:
        prediction = predictor.predict(imageURL= plantes_sc_labelled['photo_fleur'].iloc[iloc],
                        organs = "auto",
                        includeRelatedImages=False)
    except Exception as e:
        print(f"Error: {e}")
        if api_key_index + 1 < len(API_keys):
            return get_plant_details(iloc, api_key_index + 1)
        else:
            print("You used all your API keys for today.")
            return None

    famille = prediction["results"][0]["species"]["family"]["scientificNameWithoutAuthor"] # famille
    genre = prediction["results"][0]["species"]["genus"]["scientificNameWithoutAuthor"] # genre
    espece = prediction["results"][0]["species"]["scientificNameWithoutAuthor"] # espece

    return [famille, genre, espece]

In [48]:
# appelle la fonction call API sur les 10 premières lignes de plantes_sc_labelled et affiche toutes les sorties
for i in range(10):
    print(call_API(i))

['Fabaceae', 'Trifolium', 'Trifolium pratense']
['Acanthaceae', 'Acanthus', 'Acanthus spinosus']
['Orchidaceae', 'Himantoglossum', 'Himantoglossum hircinum']
['Theaceae', 'Camellia', 'Camellia sasanqua']
['Asteraceae', 'Bellis', 'Bellis sylvestris']
['Ericaceae', 'Erica', 'Erica x darleyensis']
['Fabaceae', 'Adenocarpus', 'Adenocarpus foliolosus']
['Asteraceae', 'Taraxacum', 'Taraxacum palustre']
['Brassicaceae', 'Sinapis', 'Sinapis arvensis']
['Asteraceae', 'Senecio', 'Senecio inaequidens']


In [50]:
plantes_sc_labelled.head()

,collection_id,plante_sc,Famille,Genre,Espece,plante_fr,plante_precision,plante_inconnue,plante_caractere,photo_fleur,photo_plante,photo_feuille
0,1,Les Trèfles à fleurs blanches ou roses en boul...,Fabaceae,Trifolium,Trifolium pratense,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
2,2,Acanthus mollis,NaN,NaN,NaN,Acanthe molle,NaN,NaN,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
5,3,"Les Orchidées à fleurs blanches, jaunes ou ver...",NaN,NaN,NaN,NaN,orchis bouc - Himantoglossum hircinum,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
6,4,Les Vipérines à fleurs bleues-violacées (des e...,NaN,NaN,NaN,NaN,Theaceae,0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
10,5,Les Pâquerettes (Bellis sp),NaN,NaN,NaN,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...


In [58]:
from tqdm import tqdm

def remplir_tableau(df, num_rows):
    df = df.reset_index(drop=True)
    # parcours toutes les lignes de df et appelle la fonction call_API sur chaque ligne vide et stocke les résultats dans les colonnes "Famille", "Genre", et "Espèce"
    for i in tqdm(range(num_rows)):
        if pd.isnull(df.loc[i, "Famille"]) or pd.isnull(df.loc[i, "Genre"]) or pd.isnull(df.loc[i, "Espece"]):
            try:
                df.loc[i, ["Famille", "Genre", "Espece"]] = call_API(i)
            except Exception as e:
                print(f"Error: {e}")
                continue
    return df

remplir_tableau(plantes_sc_labelled, 8)

100%|██████████| 8/8 [00:20<00:00,  2.59s/it]


,collection_id,plante_sc,Famille,Genre,Espece,plante_fr,plante_precision,plante_inconnue,plante_caractere,photo_fleur,photo_plante,photo_feuille
0,1,Les Trèfles à fleurs blanches ou roses en boul...,Fabaceae,Trifolium,Trifolium pratense,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
1,2,Acanthus mollis,Acanthaceae,Acanthus,Acanthus spinosus,Acanthe molle,NaN,NaN,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
2,3,"Les Orchidées à fleurs blanches, jaunes ou ver...",Orchidaceae,Himantoglossum,Himantoglossum hircinum,NaN,orchis bouc - Himantoglossum hircinum,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
3,4,Les Vipérines à fleurs bleues-violacées (des e...,Theaceae,Camellia,Camellia sasanqua,NaN,Theaceae,0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
4,5,Les Pâquerettes (Bellis sp),Asteraceae,Bellis,Bellis sylvestris,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
...,...,...,...,...,...,...,...,...,...,...,...,...
2615,75 468,Arbutus andrachne,NaN,NaN,NaN,Arbousier de Chypre,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
2616,75 525,Pavonia hastata,NaN,NaN,NaN,Pavonie (Pavonia hastata),NaN,NaN,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
2617,75 554,Medicago falcata,NaN,NaN,NaN,Luzerne en faucille,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
2618,75 625,Helianthus divaricatus,NaN,NaN,NaN,Hélianthe à feuilles Ètalées,NaN,NaN,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...


In [61]:
# affiche les 10 premières lignes de plantes_sc_labelled
plantes_sc_labelled.head(10)


,collection_id,plante_sc,Famille,Genre,Espece,plante_fr,plante_precision,plante_inconnue,plante_caractere,photo_fleur,photo_plante,photo_feuille
0,1,Les Trèfles à fleurs blanches ou roses en boul...,Fabaceae,Trifolium,Trifolium pratense,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
2,2,Acanthus mollis,NaN,NaN,NaN,Acanthe molle,NaN,NaN,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
5,3,"Les Orchidées à fleurs blanches, jaunes ou ver...",NaN,NaN,NaN,NaN,orchis bouc - Himantoglossum hircinum,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
6,4,Les Vipérines à fleurs bleues-violacées (des e...,NaN,NaN,NaN,NaN,Theaceae,0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
10,5,Les Pâquerettes (Bellis sp),NaN,NaN,NaN,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
24,6,Les Bruyères à fleurs roses en grappes (des es...,NaN,NaN,NaN,NaN,Bruyère carnée (Erica carnea),0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
25,7,Les Ajoncs (des espèces du genre Ulex),NaN,NaN,NaN,NaN,Ajonc (Ulex europaeus),0.0,ne se prononce pas,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
41,12,Les Pissenlits (Taraxacum sp),NaN,NaN,NaN,NaN,Pissenlit (Taraxacum officinale),0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
47,15,Les Crucifères jaunes (des espèces de la famil...,NaN,NaN,NaN,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...
54,17,Senecio jacobaea,NaN,NaN,NaN,Le Sénéçon jacobée,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...


In [60]:
plantes_sc_labelled.shape

(2620, 12)